In [3]:
# Test Zone:
# WIP

In [5]:
import pandas as pd
import numpy as np

# Plots and Graphs:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import scikitplot as skplt 
%matplotlib inline

import geopandas as gpd
import folium 

import requests
from IPython.display import Image, display

import random

import urllib.request

import warnings
warnings.filterwarnings('ignore')

# Shows all columns
pd.set_option('display.max_columns', None)

In [6]:
df = pd.read_csv('DNR_Fire_Statistics_2008_-_Present.csv')
df.head()

,X,Y,OBJECTID,FIREEVENT_ID,INCIDENT_NO,INCIDENT_NM,INCIDENT_ID,COUNTY_LABEL_NM,FIRE_TWP_WHOLE_NO,FIRE_TWP_FRACT_NO,FIRE_RGE_WHOLE_NO,FIRE_RGE_FRACT_NO,FIRE_RGE_DIR_FLG,FIRE_SECT_NO,SITE_ELEV,FIREGCAUSE_LABEL_NM,FIRESCAUSE_LABEL_NM,BURNESCAPE_RSN_LABEL_NM,ACRES_BURNED,START_DT,START_TM,DSCVR_DT,DSCVR_TM,CONTROL_DT,CONTROL_TM,FIRE_OUT_DT,FIRE_OUT_TM,BURN_MERCH_AREA,BURN_REPROD_AREA,BURN_NONSTOCK_AREA,FIREEVNT_CLASS_CD,FIREEVNT_CLASS_LABEL_NM,SECTION_SUBDIV_PTS_ID,LAT_COORD,LON_COORD,RES_ORDER_NO,NON_DNR_RES_ORDER_NO,START_OWNER_AGENCY_NM,START_JURISDICTION_AGENCY_NM,PROTECTION_TYPE,REGION_NAME
0,-1.333567e+07,6.191981e+06,1,49996,26,LITTLE PEACOCK,49829,OKANOGAN,35,0,24,0,E,26,2927.0,Recreation,Camper,NaN,0.01,2017/05/28 08:00:00+00,1800,2017/05/29 00:00:00+00,1335.0,2017/05/29 00:00:00+00,1420.0,2017/06/21 00:00:00+00,1328.0,NaN,NaN,NaN,1,Classified,662491,48.509150,-119.796370,NaN,NaN,DNR,DNR,DNR Protection FFPA,NORTHEAST
1,-1.346035e+07,5.765132e+06,2,50035,7,Turkey Ranch,49868,KLICKITAT,5,0,15,0,E,22,2000.0,Debris Burn,None,Extinguish,0.25,2017/05/23 08:00:00+00,1715,2017/05/23 00:00:00+00,1650.0,2017/05/23 00:00:00+00,1935.0,2017/05/25 00:00:00+00,1300.0,NaN,NaN,0.25,1,Classified,372894,45.904947,-120.916377,WA-SES-050,NaN,Private,DNR,DNR Protection FFPA,SOUTHEAST
2,-1.364323e+07,5.913875e+06,4,5021,90,1050 Fire,5163,THURSTON,16,0,2,0,E,33,350.0,Lightning,None,NaN,9.68,2008/08/17 08:00:00+00,300,2008/08/17 00:00:00+00,300.0,2008/08/17 00:00:00+00,1210.0,2008/09/15 00:00:00+00,1500.0,NaN,9.68,NaN,1,Classified,435486,46.826950,-122.559250,WA-PCS-0090,NaN,Private,DNR,DNR Protection FFPA,PACIFIC CASC
3,-1.306397e+07,6.094150e+06,5,7882,104,BEAR LAKE,8024,SPOKANE,28,0,43,0,E,15,1800.0,Lightning,None,NaN,0.10,2009/06/17 08:00:00+00,2015,2009/06/17 00:00:00+00,2030.0,2009/06/18 00:00:00+00,1221.0,2009/07/02 00:00:00+00,1234.0,0.1,NaN,NaN,1,Classified,552562,47.923560,-117.355620,NaN,NaN,Other Government,DNR,DNR Protection FFPA,NORTHEAST
4,-1.368587e+07,5.932888e+06,6,47157,68,HWY 101,46930,THURSTON,17,0,2,0,W,21,170.0,Recreation,Camper,NaN,0.01,2016/07/25 08:00:00+00,1730,2016/07/25 00:00:00+00,1730.0,2016/07/25 00:00:00+00,1815.0,2016/07/26 00:00:00+00,1530.0,NaN,NaN,NaN,1,Classified,560860,46.943690,-122.942240,WA-SPS-0097,NaN,Private,DNR,DNR Protection FFPA,SO PUGET


In [10]:
df_test = df[['INCIDENT_NM', 'START_DT', 'ACRES_BURNED', 'LAT_COORD', 'LON_COORD', 'SITE_ELEV', 'FIREGCAUSE_LABEL_NM']]

# Renaming columns
df_test.rename(columns={"INCIDENT_NM": "name",
                        "START_DT": "date",
                        "ACRES_BURNED": "acres", 
                        "LAT_COORD": "lat",
                        "LON_COORD": "lon",
                        "SITE_ELEV": "elevation",
                        "FIREGCAUSE_LABEL_NM" : "cause"}, inplace = True)

# Formatting the date
df_test['date'] = pd.to_datetime(df_test.date)
df_test.date = df_test.date.dt.strftime('%m/%d/%Y')
df_test['date'] = pd.to_datetime(df_test.date)

# Rounding coordinates to 6 places: (G Maps can't distinguish past 6 decimal places) 
df_test['lat'] = df_test['lat'].map(lambda x: round(x, 6))
df_test['lon'] = df_test['lon'].map(lambda x: round(x, 6))

# For center coordinates in the URL
df_test['lat'] = df_test['lat'].map(lambda x: str(x))
df_test['lon'] = df_test['lon'].map(lambda x: str(x))

# Displaying the new head
df_test.head()

,name,date,acres,lat,lon,elevation,cause
0,LITTLE PEACOCK,2017-05-28,0.01,48.50915,-119.79637,2927.0,Recreation
1,Turkey Ranch,2017-05-23,0.25,45.904947,-120.916377,2000.0,Debris Burn
2,1050 Fire,2008-08-17,9.68,46.82695,-122.55925,350.0,Lightning
3,BEAR LAKE,2009-06-17,0.10,47.92356,-117.35562,1800.0,Lightning
4,HWY 101,2016-07-25,0.01,46.94369,-122.94224,170.0,Recreation


In [11]:
# Future Goal:
# Use regression modeling techniques and above data to predict size of fire.
# Perhaps use buckets and classification instead. . . 